In [1]:
import pandas as pd
import numpy as np
import os

# ---------------------------
# CARGA DE ARCHIVO
# ---------------------------
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):
        df = pd.read_excel(path, header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/FPDEVANADO.xlsx


In [2]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
df["SERIE"] = df["SERIE"].astype(str)
df.columns = df.columns.str.replace(r"\n", " ", regex=True).str.strip()
df = df.dropna(axis=1, how="all")

# Seleccionar columnas que terminan en %
cols_fp = ["SERIE", "FECHA"] + [c for c in df.columns if c.endswith("%")]
df_fp = df[cols_fp]

# Columna con el máximo valor de FPDEVANADO
df_fp['Max_FP'] = df_fp[[c for c in df_fp.columns if c.endswith('%')]].max(axis=1)
df_fp.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_43628\4236803651.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp['Max_FP'] = df_fp[[c for c in df_fp.columns if c.endswith('%')]].max(axis=1)


,SERIE,FECHA,H ICH+ICHL %,H ICH %,H ICHL %,H ICHT %,X ICL+ICLT %,X ICL %,X ICLT %,X ICLH %,Y ICT+ICTH %,Y ICT %,Y ICTH %,Y ICTL %,Max_FP
0,59571,2016-10-23,NaN,0.260,0.200,NaN,NaN,0.210,NaN,NaN,0.18,0.14,0.19,NaN,0.260
1,59571,2006-01-29,0.22,0.230,0.170,NaN,NaN,0.190,NaN,NaN,0.17,0.13,0.17,NaN,0.230
2,59571,1996-04-16,0.12,0.122,0.104,NaN,NaN,0.139,NaN,NaN,NaN,NaN,NaN,NaN,0.139
3,59572,2016-09-18,NaN,0.210,0.150,NaN,NaN,0.180,NaN,NaN,0.15,0.11,0.15,NaN,0.210
4,59572,2006-01-29,0.21,0.210,0.150,NaN,NaN,0.170,NaN,NaN,0.17,0.12,0.15,NaN,0.210


In [3]:
# ---------------------------
# CONFIGURACIÓN DE PUNTAJE
# ---------------------------
pesos = [5, 3, 1]
limites = {"FPDEVANADO": [1.0, 0.5]}

def asignar_puntaje(df, columna_valor, columna_puntaje, limites, pesos):
    df[columna_puntaje] = np.select(
        [
            df[columna_valor] > limites[0],
            (df[columna_valor] > limites[1]) & (df[columna_valor] <= limites[0]),
            df[columna_valor] <= limites[1]
        ],
        pesos,
        default=np.nan
    )
    return df

df_fp = asignar_puntaje(df_fp, 'Max_FP', 'FPDEVANADO', limites["FPDEVANADO"], pesos)
df_fp.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_43628\2545593978.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna_puntaje] = np.select(


,SERIE,FECHA,H ICH+ICHL %,H ICH %,H ICHL %,H ICHT %,X ICL+ICLT %,X ICL %,X ICLT %,X ICLH %,Y ICT+ICTH %,Y ICT %,Y ICTH %,Y ICTL %,Max_FP,FPDEVANADO
0,59571,2016-10-23,NaN,0.260,0.200,NaN,NaN,0.210,NaN,NaN,0.18,0.14,0.19,NaN,0.260,1.0
1,59571,2006-01-29,0.22,0.230,0.170,NaN,NaN,0.190,NaN,NaN,0.17,0.13,0.17,NaN,0.230,1.0
2,59571,1996-04-16,0.12,0.122,0.104,NaN,NaN,0.139,NaN,NaN,NaN,NaN,NaN,NaN,0.139,1.0
3,59572,2016-09-18,NaN,0.210,0.150,NaN,NaN,0.180,NaN,NaN,0.15,0.11,0.15,NaN,0.210,1.0
4,59572,2006-01-29,0.21,0.210,0.150,NaN,NaN,0.170,NaN,NaN,0.17,0.12,0.15,NaN,0.210,1.0


In [4]:
# ---------------------------
# TABLAS BASE
# ---------------------------
df_full = df_fp.drop(columns='Max_FP')              # detallada (SERIE, FECHA, %, FPDEVANADO)
df_FP = df_fp[['SERIE', 'FECHA', 'FPDEVANADO']].copy()      # original (SERIE, FECHA, FPDEVANADO)

In [5]:
# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

In [6]:
# ---------- Tabla extendida FPDEVANADO ----------
ultimos_2024 = df_FP[df_FP['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_FP, ultimos_2024], ignore_index=True)

df_extendida_FP = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida_FP = df_extendida_FP.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla detallada extendida ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)

df_detalles_ext_FP = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext_FP = df_detalles_ext_FP.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

C:\Users\roquispec\AppData\Local\Temp\ipykernel_43628\3507473575.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_extendida_FP = df_extendida_FP.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_43628\3507473575.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_detalles_ext_FP = df_detalles_ext_FP.groupby("SERIE").apply(lambda g: g.ffill()).

In [7]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_FP():
    return df_FP

def get_df_extendida_FP():
    return df_extendida_FP

def get_df_detalles_FP():
    return df_full

def get_df_detalles_ext_FP():
    return df_detalles_ext_FP

# ---------------------------
# DEMO DE RESULTADOS
# ---------------------------
print("\n ====== TABLA FPDEVANADO ORIGINAL ====== \n")
print(get_df_FP().tail(), "\n")

print("\n ====== TABLA FPDEVANADO EXTENDIDA ====== \n")
print(get_df_extendida_FP().head(), "\n")

print("\n ====== TABLA DETALLADA FPDEVANADO ====== \n")
print(get_df_detalles_FP().head(), "\n")

print("\n ====== TABLA DETALLADA EXTENDIDA FPDEVANADO ====== \n")
print(get_df_detalles_ext_FP().tail(), "\n")


 ====== TABLA FPDEVANADO ORIGINAL ====== 

            SERIE      FECHA  FPDEVANADO
383  5337PA196-03 2025-01-19         NaN
384       D518294 2025-06-05         NaN
385       D518294 2025-06-05         NaN
386        359111 2022-12-14         NaN
387       L-30230 2023-11-02         NaN 


 ====== TABLA FPDEVANADO EXTENDIDA ====== 

    SERIE      FECHA  FPDEVANADO
0  100138 2015-01-01         NaN
1  100138 2015-01-02         NaN
2  100138 2015-01-03         NaN
3  100138 2015-01-04         NaN
4  100138 2015-01-05         NaN 


 ====== TABLA DETALLADA FPDEVANADO ====== 

   SERIE      FECHA  H ICH+ICHL %  H ICH %  H ICHL %  H ICHT %  X ICL+ICLT %  \
0  59571 2016-10-23           NaN    0.260     0.200       NaN           NaN   
1  59571 2006-01-29          0.22    0.230     0.170       NaN           NaN   
2  59571 1996-04-16          0.12    0.122     0.104       NaN           NaN   
3  59572 2016-09-18           NaN    0.210     0.150       NaN           NaN   
4  59572 2006-01-2